# What makes a Kickstarter campaign successful?

## Project Overview

Purpose of the project...

Data source...

## Data Cleaning

Information on how the data was cleaned...

In [2]:
# Data Cleaning Code/link to the file

## Exploratory Data Analysis

Purpose of the data exploration...

In [3]:
# Put Visualization #1 code here: 

Analysis of the visualization here...

## Statistical Modeling

In [4]:
# Statistical Modeling code here: 
import pandas as pd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%store -r data
%store -r le
newdata = data
newdata.state = data.state.astype(int)
newdata

### Multiple Regression

In [ ]:
# Creates Model
full_model = smf.ols(formula='state ~ goal + pledged + category + main_category + backers + country', data=newdata).fit()
full_model.summary()

In [ ]:
# Make predictions
data['mult_predictions'] = full_model.predict()
data.loc[data["mult_predictions"] >= 0.5,"mult_predictions"] = 1
data.loc[data["mult_predictions"] < 0.5,"mult_predictions"] = 0

In [1]:
# Visualize results
real_success = len(data[data['state'] == 1])
fake_success = len(data[data['mult_predictions'] == 1])
real_failed = len(data[data['state'] == 0])
fake_failed = len(data[data['mult_predictions'] == 0])

x = ['Pass', 'Fail']
real = [real_success, real_failed]
fake = [fake_success, fake_failed]

fig, ax = plt.subplots()
bar_width = 0.35
opacity = 0.4
N = 2
ind = np.arange(N)

rects1 = ax.bar(ind, real, bar_width, alpha=opacity, color='b', label='Real Data')
rects2 = ax.bar(ind + bar_width, fake, bar_width, alpha=opacity, color='r', label='Prediction Data')
ax.set_xlabel('True or False')
ax.set_ylabel('Total Count')
ax.set_title('Which are True vs False')
ax.set_xticks(ind + bar_width / 2)
ax.set_xticklabels(('Success', 'Failed'))
ax.legend()

plt.show

NameError: name 'data' is not defined

In [ ]:
# Percent of correct predictions
len(data[data["mult_predictions"]== data["state"]]) / len(data["state"])

## Machine Learning

In [5]:
# Machine Learning code here: 

Explanation of the machine learning done...

## *About the Team*

Information on the Team...